<a href="https://colab.research.google.com/github/mehreen89/DataSets/blob/main/mehreen_NLP_Alexa_NLTK_Hw4_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### "Amazon-Alexa" text classification using custom trained Word Embeddings ( w/ gensim )

#### https://www.kaggle.com/datasets/sid321axn/amazon-alexa-reviews

### import the required libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [ ]:
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Load the input data ( "amazon alexa reviews data")

In [ ]:
# Loading TSV file
df_amazon = pd.read_csv ("/content/sample_data/amazon_alexa.tsv", sep="\t")

In [ ]:
# Top 5 records
df_amazon.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [ ]:
df_amazon_mod = df_amazon[['verified_reviews','feedback']]
df_amazon_mod.head()

,verified_reviews,feedback
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1


### Create the tokens directly from the NLTK library

In [ ]:
import re
def clean(string):
    string = str(string)
    cleanString = re.sub('[^A-Za-z]+',' ', string )
    return cleanString

df_amazon_mod['reviews_non_numeric'] = df_amazon_mod['verified_reviews'].map(clean)
df_amazon_mod['reviews_len_trim'] = df_amazon_mod['reviews_non_numeric'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
df_amazon_mod['reviews_Tokenized'] = df_amazon_mod['reviews_len_trim'].map(lambda x: x.lower().split())
df_amazon_mod['reviews_Tokenized_stop'] = df_amazon_mod['reviews_Tokenized'].map(lambda x: " ".join(x for x in x if x not in stop_words))
df_amazon_mod['reviews_Tokenized_stop'] = df_amazon_mod['reviews_Tokenized_stop'].map(lambda x: x.split())

<ipython-input-6-8bc305872628>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_amazon_mod['reviews_non_numeric'] = df_amazon_mod['verified_reviews'].map(clean)
<ipython-input-6-8bc305872628>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_amazon_mod['reviews_len_trim'] = df_amazon_mod['reviews_non_numeric'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))


In [ ]:
df_amazon_mod

,verified_reviews,feedback,reviews_non_numeric,reviews_len_trim,reviews_Tokenized,reviews_Tokenized_stop
0,Love my Echo!,1,Love my Echo,Love Echo,"[love, echo]","[love, echo]"
1,Loved it!,1,Loved it,Loved,[loved],[loved]
2,"Sometimes while playing a game, you can answer...",1,Sometimes while playing a game you can answer ...,Sometimes while playing game you can answer qu...,"[sometimes, while, playing, game, you, can, an...","[sometimes, playing, game, answer, question, c..."
3,I have had a lot of fun with this thing. My 4 ...,1,I have had a lot of fun with this thing My yr ...,have had lot fun with this thing old learns ab...,"[have, had, lot, fun, with, this, thing, old, ...","[lot, fun, thing, old, learns, dinosaurs, cont..."
4,Music,1,Music,Music,[music],[music]
...,...,...,...,...,...,...
3145,"Perfect for kids, adults and everyone in betwe...",1,Perfect for kids adults and everyone in between,Perfect for kids adults and everyone between,"[perfect, for, kids, adults, and, everyone, be...","[perfect, kids, adults, everyone]"
3146,"Listening to music, searching locations, check...",1,Listening to music searching locations checkin...,Listening music searching locations checking t...,"[listening, music, searching, locations, check...","[listening, music, searching, locations, check..."
3147,"I do love these things, i have them running my...",1,I do love these things i have them running my ...,love these things have them running entire hom...,"[love, these, things, have, them, running, ent...","[love, things, running, entire, home, lights, ..."
3148,Only complaint I have is that the sound qualit...,1,Only complaint I have is that the sound qualit...,Only complaint have that the sound quality isn...,"[only, complaint, have, that, the, sound, qual...","[complaint, sound, quality, great, mostly, use..."


In [ ]:
gensim_custom_model = Word2Vec(df_amazon_mod['reviews_Tokenized_stop'],
                 min_count=1, # ignores words that appear less than min_count
                vector_size = 50,   # Dimensionality of words embeddings
                workers = 1,  # Number of processors ( for parallelization )
                window = 5,   # Context window for words during training
                epochs=30)      # Number of epochs training over corpus
# summarize the loaded model
print(gensim_custom_model)
# summarize vocabulary
words = gensim_custom_model.wv.index_to_key
#print(words)
# access vector for one word
#print(gensim_custom_model['sentence'])
# save model
gensim_custom_model.save('gensim_embed_model.bin')
# load model
new_model = Word2Vec.load('gensim_embed_model.bin')
print(new_model)

Word2Vec<vocab=3765, vector_size=50, alpha=0.025>
Word2Vec<vocab=3765, vector_size=50, alpha=0.025>


In [ ]:
print(len(new_model.wv['playing']))
print(new_model.wv['playing'])

50
[ 0.78717935  0.16840449  0.7126594  -0.89498276 -1.3458104  -0.810538
  0.6815014   1.4179074  -1.0823448   0.27393514  0.82063264 -0.2661764
  0.41594738 -0.5365176  -0.7125858  -0.65601474 -0.12776999 -0.31802368
 -1.9977117   1.2474657  -0.00243002  0.36826056  2.4077554   0.2639535
  1.475084   -0.6894396   0.9504755  -1.4255631  -0.02439035  1.3017248
  0.6521574  -0.59433717 -0.99086356 -0.7408575   1.3314383   0.38084903
 -1.2946125   0.8570361  -1.1895643  -1.2608664  -0.16000046  0.16396819
 -0.72046363  0.24157909  1.431961    1.0256616   0.87006253  0.0743829
 -0.747492   -0.38648608]


In [ ]:
# Display the words that are most relevant
new_model.wv.most_similar('playing')

[('personally', 0.8825592398643494),
 ('listening', 0.8610967397689819),
 ('timers', 0.8508184552192688),
 ('play', 0.8313484191894531),
 ('pandora', 0.8237659335136414),
 ('videos', 0.8235621452331543),
 ('photos', 0.8134161829948425),
 ('lyrics', 0.8000072240829468),
 ('books', 0.791361391544342),
 ('reminding', 0.7887201309204102)]

### Aggregted sentence vector for each sentence based on the word vectors

In [ ]:
words = set(new_model.wv.index_to_key)
#df_amazon_mod['reviews_Vect'] = np.array([np.array([new_model.wv[i] for i in ls if i in words])for ls in df_amazon_mod['reviews_Tokenized']])

In [ ]:
df_amazon_mod['reviews_vect'] = np.array([np.array([new_model.wv[i] for i in ls if i in words])
                       for ls in df_amazon_mod['reviews_Tokenized_stop']], dtype=object)

In [ ]:
df_amazon_mod.head()

,verified_reviews,feedback,reviews_non_numeric,reviews_len_trim,reviews_Tokenized,reviews_Tokenized_stop,reviews_vect
0,Love my Echo!,1,Love my Echo,Love Echo,"[love, echo]","[love, echo]","[[0.56772953, -0.17090826, -0.16753213, 0.5610..."
1,Loved it!,1,Loved it,Loved,[loved],[loved],"[[0.14756915, -0.34833315, -0.39622784, 0.1988..."
2,"Sometimes while playing a game, you can answer...",1,Sometimes while playing a game you can answer ...,Sometimes while playing game you can answer qu...,"[sometimes, while, playing, game, you, can, an...","[sometimes, playing, game, answer, question, c...","[[0.6506556, -0.4029045, 0.08085703, -0.198803..."
3,I have had a lot of fun with this thing. My 4 ...,1,I have had a lot of fun with this thing My yr ...,have had lot fun with this thing old learns ab...,"[have, had, lot, fun, with, this, thing, old, ...","[lot, fun, thing, old, learns, dinosaurs, cont...","[[0.6948729, 0.51839334, -0.63067174, -1.25660..."
4,Music,1,Music,Music,[music],[music],"[[1.603438, 1.6430076, 0.5169026, -1.236823, 0..."


In [ ]:
vector_size_n_w2v = 50
text_vect_avg = []
for v in df_amazon_mod['reviews_vect']:
    if v.size:
        text_vect_avg.append(v.mean(axis=0))
    else:
        text_vect_avg.append(np.zeros(vector_size_n_w2v, dtype=float)) # the same vector size must be used here as for model training


df_amazon_mod['reviews_vect_avg'] = text_vect_avg
df_amazon_mod.head()

,verified_reviews,feedback,reviews_non_numeric,reviews_len_trim,reviews_Tokenized,reviews_Tokenized_stop,reviews_vect,reviews_vect_avg
0,Love my Echo!,1,Love my Echo,Love Echo,"[love, echo]","[love, echo]","[[0.56772953, -0.17090826, -0.16753213, 0.5610...","[0.16461937, 0.17480883, -0.6616313, 0.3398885..."
1,Loved it!,1,Loved it,Loved,[loved],[loved],"[[0.14756915, -0.34833315, -0.39622784, 0.1988...","[0.14756915, -0.34833315, -0.39622784, 0.19884..."
2,"Sometimes while playing a game, you can answer...",1,Sometimes while playing a game you can answer ...,Sometimes while playing game you can answer qu...,"[sometimes, while, playing, game, you, can, an...","[sometimes, playing, game, answer, question, c...","[[0.6506556, -0.4029045, 0.08085703, -0.198803...","[0.35474572, 0.08274216, -0.40144116, 0.239658..."
3,I have had a lot of fun with this thing. My 4 ...,1,I have had a lot of fun with this thing My yr ...,have had lot fun with this thing old learns ab...,"[have, had, lot, fun, with, this, thing, old, ...","[lot, fun, thing, old, learns, dinosaurs, cont...","[[0.6948729, 0.51839334, -0.63067174, -1.25660...","[0.404853, 0.3074037, -0.059970673, -0.0973616..."
4,Music,1,Music,Music,[music],[music],"[[1.603438, 1.6430076, 0.5169026, -1.236823, 0...","[1.603438, 1.6430076, 0.5169026, -1.236823, 0...."


In [ ]:
df_Machine_Learning = pd.DataFrame(text_vect_avg)
df_Machine_Learning

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.164619,0.174809,-0.661631,0.339889,0.487158,-0.446391,0.593795,0.255107,-0.226417,0.434712,...,0.676662,0.468720,-0.213200,-0.046821,1.995222,0.594122,-1.005512,-0.777504,-0.200483,0.883136
1,0.147569,-0.348333,-0.396228,0.198844,0.639942,-0.158397,0.655995,0.760992,-0.590657,-0.295233,...,1.187615,0.079169,-0.188779,0.221683,0.852257,0.400829,-0.339623,-0.475112,0.637259,0.328914
2,0.354746,0.082742,-0.401441,0.239658,-0.416264,-0.938241,0.847596,1.147889,-0.971648,0.405093,...,0.543857,0.169328,0.026552,-0.119901,0.942708,-0.115403,0.439030,0.152681,0.327881,0.047225
3,0.404853,0.307404,-0.059971,-0.097362,-0.435646,-0.714205,0.665746,1.003516,-1.019414,0.093689,...,0.310009,0.108911,-0.201268,0.061702,1.004266,0.175111,0.540215,-0.209909,0.186223,-0.145288
4,1.603438,1.643008,0.516903,-1.236823,0.007486,-0.660388,0.220519,1.329235,-2.766103,2.242153,...,-0.888932,-0.725362,-0.597956,1.297681,1.083152,-0.027384,0.770326,-0.753851,-0.216711,-0.893886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,0.109022,-0.049810,-0.359774,0.128009,-0.043868,-0.329911,0.320737,0.899614,-0.492318,-0.008079,...,0.697675,0.032334,-0.054076,0.148231,0.787602,0.333009,0.100560,-0.294940,0.344646,0.080129
3146,0.544876,0.599447,0.264817,-0.077751,-0.791947,-0.396754,0.778613,0.553801,-1.409511,0.129346,...,0.218900,0.341273,-0.128609,-0.162773,1.540330,-0.544627,0.485739,-0.451743,-0.124086,-0.034657
3147,-0.104189,0.429585,-0.426398,0.295406,-0.189069,-0.797247,0.355355,0.792691,-0.752499,-0.031196,...,0.496167,0.019662,0.027088,0.187338,0.934472,-0.040215,0.258691,0.009890,0.553511,0.181025
3148,0.024274,-0.015226,-0.242895,-0.165325,0.301240,-0.644128,0.522627,0.846130,-0.511512,-0.173056,...,0.195328,-0.169304,-0.134241,0.125048,0.938671,0.474954,-0.104377,-0.341227,0.526719,0.110913


In [ ]:
df_Machine_Learning.columns = ['Col_' + str(i+1) for i in range(0, df_Machine_Learning.shape[1])]
df_Machine_Learning

,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,...,Col_41,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50
0,0.164619,0.174809,-0.661631,0.339889,0.487158,-0.446391,0.593795,0.255107,-0.226417,0.434712,...,0.676662,0.468720,-0.213200,-0.046821,1.995222,0.594122,-1.005512,-0.777504,-0.200483,0.883136
1,0.147569,-0.348333,-0.396228,0.198844,0.639942,-0.158397,0.655995,0.760992,-0.590657,-0.295233,...,1.187615,0.079169,-0.188779,0.221683,0.852257,0.400829,-0.339623,-0.475112,0.637259,0.328914
2,0.354746,0.082742,-0.401441,0.239658,-0.416264,-0.938241,0.847596,1.147889,-0.971648,0.405093,...,0.543857,0.169328,0.026552,-0.119901,0.942708,-0.115403,0.439030,0.152681,0.327881,0.047225
3,0.404853,0.307404,-0.059971,-0.097362,-0.435646,-0.714205,0.665746,1.003516,-1.019414,0.093689,...,0.310009,0.108911,-0.201268,0.061702,1.004266,0.175111,0.540215,-0.209909,0.186223,-0.145288
4,1.603438,1.643008,0.516903,-1.236823,0.007486,-0.660388,0.220519,1.329235,-2.766103,2.242153,...,-0.888932,-0.725362,-0.597956,1.297681,1.083152,-0.027384,0.770326,-0.753851,-0.216711,-0.893886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,0.109022,-0.049810,-0.359774,0.128009,-0.043868,-0.329911,0.320737,0.899614,-0.492318,-0.008079,...,0.697675,0.032334,-0.054076,0.148231,0.787602,0.333009,0.100560,-0.294940,0.344646,0.080129
3146,0.544876,0.599447,0.264817,-0.077751,-0.791947,-0.396754,0.778613,0.553801,-1.409511,0.129346,...,0.218900,0.341273,-0.128609,-0.162773,1.540330,-0.544627,0.485739,-0.451743,-0.124086,-0.034657
3147,-0.104189,0.429585,-0.426398,0.295406,-0.189069,-0.797247,0.355355,0.792691,-0.752499,-0.031196,...,0.496167,0.019662,0.027088,0.187338,0.934472,-0.040215,0.258691,0.009890,0.553511,0.181025
3148,0.024274,-0.015226,-0.242895,-0.165325,0.301240,-0.644128,0.522627,0.846130,-0.511512,-0.173056,...,0.195328,-0.169304,-0.134241,0.125048,0.938671,0.474954,-0.104377,-0.341227,0.526719,0.110913


In [ ]:
final_df = pd.concat([df_amazon_mod[['verified_reviews','feedback']], df_Machine_Learning], axis=1, sort=False)
final_df

,verified_reviews,feedback,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,...,Col_41,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50
0,Love my Echo!,1,0.164619,0.174809,-0.661631,0.339889,0.487158,-0.446391,0.593795,0.255107,...,0.676662,0.468720,-0.213200,-0.046821,1.995222,0.594122,-1.005512,-0.777504,-0.200483,0.883136
1,Loved it!,1,0.147569,-0.348333,-0.396228,0.198844,0.639942,-0.158397,0.655995,0.760992,...,1.187615,0.079169,-0.188779,0.221683,0.852257,0.400829,-0.339623,-0.475112,0.637259,0.328914
2,"Sometimes while playing a game, you can answer...",1,0.354746,0.082742,-0.401441,0.239658,-0.416264,-0.938241,0.847596,1.147889,...,0.543857,0.169328,0.026552,-0.119901,0.942708,-0.115403,0.439030,0.152681,0.327881,0.047225
3,I have had a lot of fun with this thing. My 4 ...,1,0.404853,0.307404,-0.059971,-0.097362,-0.435646,-0.714205,0.665746,1.003516,...,0.310009,0.108911,-0.201268,0.061702,1.004266,0.175111,0.540215,-0.209909,0.186223,-0.145288
4,Music,1,1.603438,1.643008,0.516903,-1.236823,0.007486,-0.660388,0.220519,1.329235,...,-0.888932,-0.725362,-0.597956,1.297681,1.083152,-0.027384,0.770326,-0.753851,-0.216711,-0.893886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,"Perfect for kids, adults and everyone in betwe...",1,0.109022,-0.049810,-0.359774,0.128009,-0.043868,-0.329911,0.320737,0.899614,...,0.697675,0.032334,-0.054076,0.148231,0.787602,0.333009,0.100560,-0.294940,0.344646,0.080129
3146,"Listening to music, searching locations, check...",1,0.544876,0.599447,0.264817,-0.077751,-0.791947,-0.396754,0.778613,0.553801,...,0.218900,0.341273,-0.128609,-0.162773,1.540330,-0.544627,0.485739,-0.451743,-0.124086,-0.034657
3147,"I do love these things, i have them running my...",1,-0.104189,0.429585,-0.426398,0.295406,-0.189069,-0.797247,0.355355,0.792691,...,0.496167,0.019662,0.027088,0.187338,0.934472,-0.040215,0.258691,0.009890,0.553511,0.181025
3148,Only complaint I have is that the sound qualit...,1,0.024274,-0.015226,-0.242895,-0.165325,0.301240,-0.644128,0.522627,0.846130,...,0.195328,-0.169304,-0.134241,0.125048,0.938671,0.474954,-0.104377,-0.341227,0.526719,0.110913


In [ ]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(df_Machine_Learning, final_df['feedback'])

LogisticRegression()

In [ ]:
import pickle as pk
pk.dump(classifier, open('clf_model.pkl', 'wb'))

In [ ]:
w2v_model_reloaded = Word2Vec.load("gensim_embed_model.bin")
vector_size_n_reloaded = 50
#vector_size_n_reloaded = pk.load(open("word2vec/vector_size_w2v_metric.pkl",'rb'))

print(w2v_model_reloaded)
print(vector_size_n_reloaded)

Word2Vec<vocab=3765, vector_size=50, alpha=0.025>
50


### Predictions on New Test data

In [ ]:
new_input = ["Flowers I like to see in the park especially sunflowers",
             "I like flowers"]

print(new_input[0])
print(new_input[1])

Flowers I like to see in the park especially sunflowers
I like flowers


In [ ]:
new_input_df = pd.DataFrame(new_input, columns=['New_Input'])
new_input_df

,New_Input
0,Flowers I like to see in the park especially s...
1,I like flowers


### Data processing

In [ ]:
import re
def clean(string):
    string = str(string)
    cleanString = re.sub('[^A-Za-z]+',' ', string )
    return cleanString

new_input_df['reviews_non_numeric'] = new_input_df['New_Input'].map(clean)
new_input_df['reviews_len_trim'] = new_input_df['reviews_non_numeric'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
new_input_df['reviews_Tokenized'] = new_input_df['reviews_len_trim'].map(lambda x: x.lower().split())
new_input_df['reviews_Tokenized_stop'] = new_input_df['reviews_Tokenized'].map(lambda x: " ".join(x for x in x if x not in stop_words))
new_input_df['reviews_Tokenized_stop'] = new_input_df['reviews_Tokenized_stop'].map(lambda x: x.split())

In [ ]:
new_input_df

,New_Input,reviews_non_numeric,reviews_len_trim,reviews_Tokenized,reviews_Tokenized_stop
0,Flowers I like to see in the park especially s...,Flowers I like to see in the park especially s...,Flowers like see the park especially sunflowers,"[flowers, like, see, the, park, especially, su...","[flowers, like, see, park, especially, sunflow..."
1,I like flowers,I like flowers,like flowers,"[like, flowers]","[like, flowers]"


In [ ]:
words = set(w2v_model_reloaded.wv.index_to_key )
new_input_df['New_Input_vect'] = np.array([np.array([w2v_model_reloaded.wv[i] for i in ls if i in words])
                                           for ls in new_input_df['reviews_Tokenized_stop']], dtype=object)


new_input_df

,New_Input,reviews_non_numeric,reviews_len_trim,reviews_Tokenized,reviews_Tokenized_stop,New_Input_vect
0,Flowers I like to see in the park especially s...,Flowers I like to see in the park especially s...,Flowers like see the park especially sunflowers,"[flowers, like, see, the, park, especially, su...","[flowers, like, see, park, especially, sunflow...","[[0.5509272, -0.023269672, -1.2413877, -0.0718..."
1,I like flowers,I like flowers,like flowers,"[like, flowers]","[like, flowers]","[[0.5509272, -0.023269672, -1.2413877, -0.0718..."


In [ ]:
text_vect_avg = []
for v in new_input_df['New_Input_vect']:
    if v.size:
        text_vect_avg.append(v.mean(axis=0))
    else:
        text_vect_avg.append(np.zeros(vector_size_n_reloaded, dtype=float)) # the same vector size must be used here as for model training


new_input_df['New_Input_vect_avg'] = text_vect_avg
new_input_df

,New_Input,reviews_non_numeric,reviews_len_trim,reviews_Tokenized,reviews_Tokenized_stop,New_Input_vect,New_Input_vect_avg
0,Flowers I like to see in the park especially s...,Flowers I like to see in the park especially s...,Flowers like see the park especially sunflowers,"[flowers, like, see, the, park, especially, su...","[flowers, like, see, park, especially, sunflow...","[[0.5509272, -0.023269672, -1.2413877, -0.0718...","[0.3059674, 0.124565676, -0.43798512, -0.03826..."
1,I like flowers,I like flowers,like flowers,"[like, flowers]","[like, flowers]","[[0.5509272, -0.023269672, -1.2413877, -0.0718...","[0.5509272, -0.023269672, -1.2413877, -0.07186..."


In [ ]:
new_input_Machine_Learning_df = pd.DataFrame(text_vect_avg)
new_input_Machine_Learning_df.columns = ['Col_' + str(i+1) for i in range(0, new_input_Machine_Learning_df.shape[1])]
new_input_Machine_Learning_df

,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,Col_10,...,Col_41,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50
0,0.305967,0.124566,-0.437985,-0.038269,-0.499309,-0.452254,0.566182,0.642892,-0.960657,0.077111,...,0.059267,0.166243,-0.274586,0.163783,0.690375,0.464710,0.313635,-0.232001,0.204953,-0.064290
1,0.550927,-0.023270,-1.241388,-0.071864,-0.079229,-1.158388,0.746383,0.258740,-0.885507,-0.288319,...,-0.482864,0.679753,0.090495,-0.419987,0.745179,0.659676,0.553713,0.153915,0.490736,0.094565


In [ ]:
final_new_input_df = pd.concat([new_input_df[['New_Input']], new_input_Machine_Learning_df], axis=1, sort=False)
final_new_input_df

,New_Input,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_41,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50
0,Flowers I like to see in the park especially s...,0.305967,0.124566,-0.437985,-0.038269,-0.499309,-0.452254,0.566182,0.642892,-0.960657,...,0.059267,0.166243,-0.274586,0.163783,0.690375,0.464710,0.313635,-0.232001,0.204953,-0.064290
1,I like flowers,0.550927,-0.023270,-1.241388,-0.071864,-0.079229,-1.158388,0.746383,0.258740,-0.885507,...,-0.482864,0.679753,0.090495,-0.419987,0.745179,0.659676,0.553713,0.153915,0.490736,0.094565


In [ ]:
clf_reloaded = pk.load(open("clf_model.pkl",'rb'))

y_pred = clf_reloaded.predict(new_input_Machine_Learning_df)
y_pred
final_new_input_df['Prediction'] = y_pred
final_new_input_df

,New_Input,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Col_9,...,Col_42,Col_43,Col_44,Col_45,Col_46,Col_47,Col_48,Col_49,Col_50,Prediction
0,Flowers I like to see in the park especially s...,0.305967,0.124566,-0.437985,-0.038269,-0.499309,-0.452254,0.566182,0.642892,-0.960657,...,0.166243,-0.274586,0.163783,0.690375,0.464710,0.313635,-0.232001,0.204953,-0.064290,1
1,I like flowers,0.550927,-0.023270,-1.241388,-0.071864,-0.079229,-1.158388,0.746383,0.258740,-0.885507,...,0.679753,0.090495,-0.419987,0.745179,0.659676,0.553713,0.153915,0.490736,0.094565,1


In [ ]:
final_new_input_df[['New_Input','Prediction']]

,New_Input,Prediction
0,Flowers I like to see in the park especially s...,1
1,I like flowers,1
